In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from datetime import datetime
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)


class GestionDatos:
    def __init__(self):
        self.nombre_archivo = "registros.xlsx"
        self.crear_dataframes()
        self.cargar_dataframes()
        self.guardar_dataframes()

    def crear_dataframes(self):
        self.columnas_clientes = ["Cedula", "Nombre", "Telefono"]
        self.columnas_productos = [
            "Referencia",
            "Codigo de barras",
            "Marca",
            "Precio de adquisicion",
            "Precio venta",
            "Unidades actuales",
            "Producto disponible",
            "Fecha",
        ]
        self.columnas_metodo_pago = ["ID", "Metodo de pago"]
        self.columnas_venta_productos = [
            "ID venta",
            "Cedula",
            "Cliente",
            "Producto",
            "Fecha",
            "Cantidad",
            "Subtotal",
            "ID_MetodoPago",
        ]
        self.columnas_venta_servicios = [
            "ID venta",
            "Cedula",
            "Cliente",
            "Servicio",
            "Fecha",
            "Cantidad",
            "Subtotal",
            "ID_MetodoPago",
        ]
        self.columnas_servicios = ["ID servicio", "Nombre Servicio", "Costo"]
        self.columnas_reservas_servicios = [
            "ID Reserva",
            "Cliente",
            "Servicio",
            "Fecha Reserva",
            "Fecha Servicio",
        ]
        self.columnas_facturas = [
            "Cedula",
            "Cliente",
            "Fecha",
            "SubtotalProductos",
            "SubtotalServicios",
            "Total",
            "Metodo de pago",
        ]
        self.columnas_usuarios = ["ID usuario", "usuario", "contraseña", "Rol ID"]
        self.columnas_roles = ["ID", "Rol"]

        self.clientes = pd.DataFrame(columns=self.columnas_clientes)
        self.productos = pd.DataFrame(columns=self.columnas_productos)
        datos_metodo_pago = {"ID": [1, 2], "Metodo de pago": ["efectivo", "tarjeta"]}
        self.metodo_pago = pd.DataFrame(
            datos_metodo_pago, columns=self.columnas_metodo_pago
        )
        self.venta_productos = pd.DataFrame(columns=self.columnas_venta_productos)
        self.venta_servicios = pd.DataFrame(columns=self.columnas_venta_servicios)
        self.servicios = pd.DataFrame(columns=self.columnas_servicios)
        self.reservas_servicios = pd.DataFrame(columns=self.columnas_reservas_servicios)
        self.facturas = pd.DataFrame(columns=self.columnas_facturas)
        self.usuarios = pd.DataFrame(columns=self.columnas_usuarios)
        datos_roles = {"ID": [1, 2, 3], "Rol": ["soporte", "admin", "caja"]}
        self.roles = pd.DataFrame(datos_roles, columns=self.columnas_roles)

        # Crear un usuario de soporte predeterminado
        usuario_soporte = pd.DataFrame(
            [
                [1, "soporte", "hola", 1],
                [2, "cajero", "cajero", 3],
                [3, "admin", "admin", 2],
            ],
            columns=self.columnas_usuarios,
        )
        self.usuarios = pd.concat([self.usuarios, usuario_soporte], ignore_index=True)

    def cargar_dataframes(self):
        try:
            archivo = pd.ExcelFile(self.nombre_archivo)
            self.clientes = pd.read_excel(archivo, sheet_name="Clientes")
            self.productos = pd.read_excel(archivo, sheet_name="Productos")
            self.metodo_pago = pd.read_excel(archivo, sheet_name="Metodo de pago")
            self.venta_productos = pd.read_excel(archivo, sheet_name="VentaProductos")
            self.venta_servicios = pd.read_excel(archivo, sheet_name="VentaServicios")
            self.servicios = pd.read_excel(archivo, sheet_name="Servicios")
            self.reservas_servicios = pd.read_excel(
                archivo, sheet_name="ReservasServicios"
            )
            self.facturas = pd.read_excel(archivo, sheet_name="Facturas")
            self.usuarios = pd.read_excel(archivo, sheet_name="Usuarios")
            self.roles = pd.read_excel(archivo, sheet_name="Roles")
        except FileNotFoundError:
            print(
                f"Archivo {self.nombre_archivo} no encontrado. Se crearán nuevos DataFrames."
            )
        except Exception as e:
            print(f"Error al cargar los DataFrames desde el archivo Excel: {e}")

    def exportar_a_json(self):
        try:
            self.clientes.to_json('clientes.json', orient='records', lines=True)
            self.productos.to_json('productos.json', orient='records', lines=True)
            self.metodo_pago.to_json('metodo_pago.json', orient='records', lines=True)
            self.venta_productos.to_json('venta_productos.json', orient='records', lines=True)
            self.venta_servicios.to_json('venta_servicios.json', orient='records', lines=True)
            self.servicios.to_json('servicios.json', orient='records', lines=True)
            self.reservas_servicios.to_json('reservas_servicios.json', orient='records', lines=True)
            self.facturas.to_json('facturas.json', orient='records', lines=True)
            self.usuarios.to_json('usuarios.json', orient='records', lines=True)
            self.roles.to_json('roles.json', orient='records', lines=True)
            print("Datos exportados a JSON exitosamente.")
        except Exception as e:
            print(f"Error al exportar los DataFrames a JSON: {e}")


    def ajustar_columnas_excel(self):
        try:
            archivo = load_workbook(self.nombre_archivo)
            for hoja in archivo.sheetnames:
                hoja_actual = archivo[hoja]
                for columnas in hoja_actual.columns:
                    max_length = 0
                    columna = [celda for celda in columnas]
                    for celda in columna:
                        try:
                            if celda.value is not None:
                                max_length = max(max_length, len(str(celda.value)))
                        except Exception as e:
                            print(f"Error al procesar la celda {celda.coordinate}: {e}")
                    ajusta_ancho = max_length + 2
                    columna_letra = get_column_letter(columna[0].column)
                    hoja_actual.column_dimensions[columna_letra].width = ajusta_ancho
            archivo.save(self.nombre_archivo)
        except Exception as e:
            print(f"Error al ajustar las columnas en el archivo Excel: {e}")

    def verificar_admin_existente(self):
        return "Admin" in self.roles["Rol"].values

    def verificar_stock_no_negativo(self):
        return all(self.productos["Unidades actuales"] >= 0)

    def realizar_verificaciones(self):
        if not self.verificar_admin_existente():
            print("Advertencia: No hay usuarios administradores en el sistema.")
        if not self.verificar_stock_no_negativo():
            print("Error: El stock no puede ser negativo en el inventario.")

    # Clientes
    def agregar_cliente(self, cedula, nombre, telefono):
        nuevo_cliente = pd.DataFrame(
            [[cedula, nombre, telefono]], columns=self.columnas_clientes
        )
        self.clientes = pd.concat([self.clientes, nuevo_cliente], ignore_index=True)
        self.guardar_dataframes()

    def buscar_cliente(self, cedula):
        cliente = self.clientes[self.clientes["Cedula"] == cedula]
        if not cliente.empty:
            return cliente
        else:
            print(f"Cliente con cedula {cedula} no encontrado.")
            return None

    def eliminar_clientes(self, cedula):
        cliente = self.clientes[self.clientes["Cedula"] == cedula]
        if not cliente.empty:
            self.clientes = self.clientes[self.clientes["Cedula"] != cedula]
            self.guardar_dataframes()
            return True
        else:
            return False

    def actualizar_cliente(self, cedula, nuevos_datos):
        cliente = self.clientes[self.clientes["Cedula"] == cedula]
        if not cliente.empty:
            for key, value in nuevos_datos.items():
                if key in self.clientes.columns:
                    self.clientes.loc[self.clientes["Cedula"] == cedula, key] = value
            self.guardar_dataframes()
            return True
        else:
            return False

    # Productos
    def agregar_producto(
        self,
        referencia,
        codigo_barras,
        marca,
        precio_adquisicion,
        precio_venta,
        unidades_actuales,
    ):
        fecha = datetime.now().strftime("%d/%m/%Y %H:%M")
        disponible = int(unidades_actuales) > 0
        nuevo_producto = pd.DataFrame(
            [
                [
                    referencia,
                    codigo_barras,
                    marca,
                    precio_adquisicion,
                    precio_venta,
                    unidades_actuales,
                    disponible,
                    fecha,
                ]
            ],
            columns=self.columnas_productos,
        )
        # self.productos = pd.concat([self.productos, nuevo_producto], ignore_index=True)
        self.guardar_dataframes()

        # Especificar los tipos de datos de las columnas
        tipos_de_datos = {
            "Referencia": str,
            "Codigo de barras": str,
            "Marca": str,
            "Precio de adquisicion": float,
            "Precio venta": float,
            "Unidades actuales": int,
            "Producto disponible": bool,
            "Fecha": str,
        }
        nuevo_producto = nuevo_producto.astype(tipos_de_datos)

        # Comprobar si el DataFrame 'nuevo_producto' no está vacío y no contiene solo valores NA
        if not nuevo_producto.empty and not nuevo_producto.isna().all().all():
            self.productos = pd.concat(
                [self.productos, nuevo_producto], ignore_index=True
            )

        self.guardar_dataframes()

    def buscar_producto(self, codigo):
        codigo = int(codigo)
        producto = self.productos[self.productos["Codigo de barras"] == codigo]
        if not producto.empty:
            return True
        else:
            return False

    def verificar_disponibilidad(self, codigo):
        producto = self.productos[self.productos["Codigo de barras"] == codigo]
        if not producto.empty:
            if producto.iloc[0]["Unidades actuales"] > 0:
                print(f"El producto con referencia {codigo} está disponible.")
            else:
                print(f"El producto con referecodigoBarrascia {codigo} no está disponible.")
        else:
            print(f"Producto con la referencia: {codigo} no ha sido encontrado")

    def actualizar_producto(self, codigoBarras, nuevos_datos):
        producto = self.productos[self.productos["Codigo de barras"] == codigoBarras]
        if not producto.empty:
            for key, value in nuevos_datos.items():
                if key in self.productos.columns:
                    self.productos.loc[
                        self.productos["Codigo de barras"] == codigoBarras, key
                    ] = value
            self.guardar_dataframes()
            #print(f"Producto con codigoBarras {codigoBarras} ha sido actualizado.")
        else:
            #print(f"Producto con codigoBarras {codigoBarras} no encontrado.")
            None

    def descontinuar_producto(self, codigo):
        if codigo in self.productos["Codigo de barras"].values:
            producto = self.productos[self.productos["Codigo de barras"] == producto]
        if not producto.empty:
            producto["Producto disponible"] = False
            self.productos = self.productos[
                self.productos["Codigo de barras"] != codigo
            ]
            pd.concat([self.productos, producto], ignore_index=True)
            self.guardar_dataframes()
            return True
        else:
            return False

    # Venta de productos y servicios
    def agregar_venta_servicio(
        self,
        id_servicio,
        cedula,
        cliente,
        nombre_servicio,
        cantidad,
        subtotal,
        ID_MetodoPago,
    ):
        fecha = datetime.now().strftime("%d/%m/%Y %H:%M")
        nueva_venta_servicio = pd.DataFrame(
            [
                [
                    id_servicio,
                    cedula,
                    cliente,
                    nombre_servicio,
                    fecha,
                    cantidad,
                    subtotal,
                    ID_MetodoPago,
                ]
            ],
            columns=self.columnas_venta_servicios,
        )
        self.venta_servicios = pd.concat(
            [self.venta_servicios, nueva_venta_servicio], ignore_index=True
        )
        self.guardar_dataframes()

    def agregar_venta_producto(
        self, id_venta, cedula, cliente, producto, cantidad, subtotal, ID_MetodoPago
    ):
        fecha = datetime.now().strftime("%d/%m/%Y %H:%M")
        nueva_venta_producto = pd.DataFrame(
            [
                [
                    id_venta,
                    cedula,
                    cliente,
                    producto,
                    fecha,
                    cantidad,
                    subtotal,
                    ID_MetodoPago,
                ]
            ],
            columns=self.columnas_venta_productos,
        )
        self.venta_productos = pd.concat(
            [self.venta_productos, nueva_venta_producto], ignore_index=True
        )
        self.guardar_dataframes()

    def actualizar_venta(self, id_venta, nuevos_datos, tipo_venta="producto"):
        if tipo_venta == "producto":
            venta = self.venta_productos[self.venta_productos["ID venta"] == id_venta]
            if not venta.empty:
                for key, value in nuevos_datos.items():
                    if key in self.venta_productos.columns:
                        self.venta_productos.loc[
                            self.venta_productos["ID venta"] == id_venta, key
                        ] = value
                self.guardar_dataframes()
                print(f"Venta de producto con ID {id_venta} actualizada.")
            else:
                print(f"Venta de producto con ID {id_venta} no encontrada.")
        elif tipo_venta == "servicio":
            venta = self.venta_servicios[self.venta_servicios["ID venta"] == id_venta]
            if not venta.empty:
                for key, value in nuevos_datos.items():
                    if key in self.venta_servicios.columns:
                        self.venta_servicios.loc[
                            self.venta_servicios["ID venta"] == id_venta, key
                        ] = value
                self.guardar_dataframes()
                print(f"Venta de servicio con ID {id_venta} actualizada.")
            else:
                print(f"Venta de servicio con ID {id_venta} no encontrada.")

    def eliminar_venta(self, id_venta, tipo_venta="producto"):
        if tipo_venta == "producto":
            venta = self.venta_productos[self.venta_productos["ID venta"] == id_venta]
            if not venta.empty:
                self.venta_productos = self.venta_productos[
                    self.venta_productos["ID venta"] != id_venta
                ]
                self.guardar_dataframes()
                print(f"El producto con el ID: {id_venta} ha sido eliminado")
            else:
                print(f"El producto con el ID: {id_venta} no ha sido encontrado")
        elif tipo_venta == "servicio":
            venta = self.venta_servicios[self.venta_servicios["ID venta"] == id_venta]
            if not venta.empty:
                self.venta_servicios = self.venta_servicios[
                    self.venta_servicios["ID venta"] != id_venta
                ]
                self.guardar_dataframes()
                print(f"El servicio con el ID: {id_venta} ha sido eliminado")
            else:
                print(f"El servicio con el ID: {id_venta} no ha sido encontrado")

    # Servicios
    def agregar_servicio(self, id_servicio, nombre_servicio, costo):
        nuevo_servicio = pd.DataFrame(
            [[id_servicio, nombre_servicio, costo]], columns=self.columnas_servicios
        )
        self.servicios = pd.concat([self.servicios, nuevo_servicio], ignore_index=True)
        self.guardar_dataframes()

    def actualizar_servicio(self, id_servicio, nuevos_datos):
        servicio = self.servicios[self.servicios['ID servicio'] == id_servicio]
        if not servicio.empty:
            for key, value in nuevos_datos.items():
                if key in self.servicios.columns:
                    self.servicios.loc[self.servicios['ID servicio'] == id_servicio, key] = value
            self.guardar_dataframes()

    def buscar_servicio(self, id_servicio):
        servicio = self.servicios[self.servicios["ID servicio"] == id_servicio]
        if not servicio.empty:
            return servicio
        else:
            print(f"Servicio con ID {id_servicio} no encontrado.")
            return None

    def eliminar_servicio(self, id_servicio):
        servicio = self.servicios[self.servicios["ID servicio"] == id_servicio]
        if not servicio.empty:
            self.servicios = self.servicios[
                self.servicios["ID servicio"] != id_servicio
            ]
            self.guardar_dataframes()
            print(f"Servicio con ID {id_servicio} eliminado.")
        else:
            print(f"Servicio con ID {id_servicio} no encontrado.")

    # Reserva de Servicios
    def reservar_servicio(self, id_reserva, cliente, servicio, fecha_servicio):
        fecha_reserva = datetime.now().strftime("%d/%m/%Y %H:%M")
        nueva_reserva = pd.DataFrame(
            [[id_reserva, cliente, servicio, fecha_reserva, fecha_servicio]],
            columns=self.columnas_reservas_servicios,
        )
        self.reservas_servicios = pd.concat(
            [self.reservas_servicios, nueva_reserva], ignore_index=True
        )
        self.guardar_dataframes()

    def cancelar_reserva_servicio(self, id_reserva):
        reserva = self.reservas_servicios[
            self.reservas_servicios["ID Reserva"] == id_reserva
        ]
        if not reserva.empty:
            self.reservas_servicios = self.reservas_servicios[
                self.reservas_servicios["ID Reserva"] != id_reserva
            ]
            self.guardar_dataframes()
        else:
            return None

    def buscar_reserva_por_servicio(self, id_servicio):
        reservas = self.reservas_servicios[
            self.reservas_servicios["Servicio"] == id_servicio
        ]
        if not reservas.empty:
            return reservas
        else:
            return None

    # Usuarios
    def agregar_usuario(self, id_usuario, usuario, contraseña, rol_id):
        nuevo_usuario = pd.DataFrame(
            [[id_usuario, usuario, contraseña, rol_id]], columns=self.columnas_usuarios
        )

        # Insertar debajo del usuario predeterminado
        indice_usuario_predeterminado = self.usuarios[
            self.usuarios["usuario"] == "soporte"
        ].index

        if not indice_usuario_predeterminado.empty:
            indice_insertar = indice_usuario_predeterminado[0] + 1
            self.usuarios = pd.concat(
                [
                    self.usuarios.iloc[:indice_insertar],
                    nuevo_usuario,
                    self.usuarios.iloc[indice_insertar:],
                ],
                ignore_index=True,
            )
        else:
            self.usuarios = pd.concat([self.usuarios, nuevo_usuario], ignore_index=True)

        self.guardar_dataframes()

    def buscar_usuario(self, usuario):
        usuarios = self.usuarios[self.usuarios["usuario"] == usuario]
        if not usuarios.empty:
            return usuarios
        else:
            print(f"El usuario {usuario} no fue encontrado")
            return None

    def modificar_usuario(self, usuario, nuevos_datos):
        usuarios = self.usuarios[self.usuarios["usuario"] == usuario]
        if not usuarios.empty:
            for key, value in nuevos_datos.items():
                if key in self.usuarios.columns:
                    self.usuarios.loc[self.usuarios["usuario"] == usuario, key] = value
            self.guardar_dataframes()
            print(f"Usuario {usuario} modificado correctamente.")
        else:
            print(f"El usuario {usuario} no ha sido encontrado.")

    # Facturas
    def conversion_factura(self, cedula, ID_MetodoPago):
        fecha = datetime.now().strftime("%d/%m/%Y %H:%M")
        try:
            cliente_servicios = self.venta_servicios[
                (self.venta_servicios["Cedula"] == cedula)
                & (self.venta_servicios["Fecha"] == fecha)
            ]
            cliente_productos = self.venta_productos[
                (self.venta_productos["Cedula"] == cedula)
                & (self.venta_productos["Fecha"] == fecha)
            ]

            if not cliente_servicios.empty:
                nombre_cliente = cliente_servicios["Cliente"].iloc[0]
            elif not cliente_productos.empty:
                nombre_cliente = cliente_productos["Cliente"].iloc[0]
            else:
                print(
                    f"No se encontraron ventas para el cliente con cédula {cedula} en la fecha {fecha}."
                )
                return

            total_venta_servicios = 0
            total_venta_productos = 0
            if not cliente_servicios.empty:
                total_venta_servicios = cliente_servicios["Subtotal"].sum()
            if not cliente_productos.empty:
                total_venta_productos = cliente_productos["Subtotal"].sum()
            total = total_venta_servicios + total_venta_productos

            nueva_factura = pd.DataFrame(
                [
                    [
                        cedula,
                        nombre_cliente,
                        fecha,
                        total_venta_productos,
                        total_venta_servicios,
                        total,
                        ID_MetodoPago,
                    ]
                ],
                columns=self.columnas_facturas,
            )
            self.facturas = pd.concat([self.facturas, nueva_factura], ignore_index=True)
            self.guardar_dataframes()
        except Exception as e:
            print(f"Error al generar la factura: {e}")


In [ ]:
from PyQt5 import uic
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QCursor
from PyQt5.QtWidgets import QMainWindow, QMessageBox, QPushButton
from GUI.ventanas.Caja import ControlNavegacion, Aplicacion
from GUI.ventanas.soporte_admin import AdminSoporteManager
from API.prueba import Cajero
from API.DATA import GestionDatos
from API.Validaciones import *
import subprocess


class Login(QMainWindow):
    def __init__(self, app):
        super().__init__()
        uic.loadUi(
            r"GUI\ui\Login.ui",
            self,
        )
        self.gestion_datos = GestionDatos()
        self.app = app
        # BT CLOSE POPUP
        self.pushButton_close_pupup.clicked.connect(lambda: self.frame_error.hide())
        self.cajero = Cajero()
        self.manual = (
            r"DESIGN\Manual_de_Uso_del_Sistema_de_Gestión_de_Glam_Makeup_Store.pdf"
        )
        self.pushBoton_manual.clicked.connect(self.abrir_manual)
        # HIDE ERROR
        self.frame_error.hide()

        # BT LOGIN
        self.pushButton_login.clicked.connect(self.checkFields)
        self.pushButton_login.clicked.connect(self.limpiarCampo)

        self.stylePopupError = (
            "background-color: rgb(255, 85, 127); border-radius: 5px;"
        )
        self.stylePopupOk = "background-color: rgb(255, 0, 0); border-radius: 5px;"

        buttons = self.findChildren(QPushButton)
        for button in buttons:
            button.setCursor(QCursor(Qt.PointingHandCursor))

    def limpiarCampo(self):
        self.lineEdit_user.setText("")
        self.lineEdit_password.setText("")

    def checkFields(self):
        username = self.lineEdit_user.text()
        password = self.lineEdit_password.text()

        def showMessage(message, is_error=True):
            self.frame_error.show()
            self.label_error.setText(message)
            if is_error:
                self.frame_error.setStyleSheet(self.stylePopupError)
            else:
                self.frame_error.setStyleSheet(self.stylePopupOk)

        # CHECK USER
        if not username:
            textUser = " Usuario Vacio. "
        else:
            textUser = ""

        # CHECK PASSWORD
        if not password:
            textPassword = " Contraseña Vacia. "
        else:
            textPassword = ""

        # CHECK FIELDS
        if textUser + textPassword != "":
            text = textUser + textPassword
            showMessage(text)
        else:
            user_role = self.authenticate_user(username, password)
            if user_role:
                if user_role == 1:
                    self.openAdminSupportWindow("soporte")
                elif user_role == 2:
                    self.openAdminSupportWindow("admin")
                elif user_role == 3:
                    self.openCajaWindow()
                else:
                    showMessage("Credenciales incorrectas")
            else:
                showMessage("Usuario o contraseña incorrecta. ")
                self.showErrorMessage("Credenciales Incorrectas")

    def showErrorMessage(self, message):
        msg_box = QMessageBox()
        msg_box.setIcon(QMessageBox.Warning)
        msg_box.setText(message)
        msg_box.setWindowTitle("Error de autenticación")
        msg_box.exec_()

    def authenticate_user(self, username, password):
        usuario_datos = self.gestion_datos.usuarios[
            self.gestion_datos.usuarios["usuario"] == username
        ]
        if not usuario_datos.empty:
            if password in usuario_datos["contraseña"].values:
                rol = usuario_datos["Rol ID"].values
                return int(rol)
            return False
        return False

    def openAdminSupportWindow(self, user_role: str):
        self.admin_soporte = AdminSoporteManager(self, user_role=user_role)
        self.admin_soporte.leer_estilos(
            self.app,
            [
                "GUI/sub_ventanas/css/admin.css",
            ],
        )
        self.admin_soporte.run()
        self.close()

    def openCajaWindow(self):
        caja = Aplicacion(self)
        caja.show()
        self.close()

    def abrir_manual(self):
        file = self.manual
        subprocess.call(["start", file], shell=True)


In [ ]:
import os
import json

from typing import List
from PyQt5 import uic
from PyQt5.QtWidgets import (
    QMainWindow,
    QApplication,
    QStackedWidget,
    QPushButton,
    QFileDialog,
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QIcon, QCursor
from PyQt5 import uic
from PyQt5.QtGui import QPixmap

from GUI.sub_ventanas.custom.utils.css import CBackground
from GUI.sub_ventanas.reportes import (
    ReportePanel, Ventas,
    Inventario
)
from GUI.sub_ventanas.reportes_diarios import (
    ReportesDiarios, ReporteDiarioCatalogo,
    ReporteDiarioProductos, ReporteDiarioVentas
)
from GUI.sub_ventanas.administrar_usuarios import AdministrarUsuarios
from GUI.sub_ventanas.inventario_productos import InventarioProductos
from GUI.sub_ventanas.GestionClientes import GestionClientes
from GUI.sub_ventanas.catalogo_servicios import GestionServicios
from GUI.sub_ventanas.cambio_contrasena import PasswordChange
from GUI.sub_ventanas.ventas import VentasAdmin
import API.DATA as GD
GD = GD.GestionDatos()

class AdminSoporte(QMainWindow, CBackground):
    def __init__(self, role: str) -> None:
        super(QMainWindow, self).__init__()
        self.role = role

        uic.loadUi(r"GUI\sub_ventanas\ui\reportes\adminDesigner.ui", self)

        self.inicializar(
            is_admin=True if self.role.strip().lower() == "admin" else False
        )

    def inicializar(self, is_admin: str | bool) -> None:
        if is_admin or is_admin == "admin":
            self.setWindowTitle("Administrador")
            self.title.setText("Admin")
            self.roleBtn.setText("Reporte\nDiario")
            self.roleBtn.setObjectName("reporteDiarioBtn")
            self.pushButton_cambiarLogo.setText("Cambiar Contraseña")
            self.pushButton_cambiarLogo.setObjectName("cambiarContraseñaBtn")
            return None

        self.setWindowTitle("Soporte")
        self.title.setText("Soporte")
        self.roleBtn.setText("Administrar\nusuario")
        self.roleBtn.setObjectName("administrarUsuarioBtn")
        self.pushButton_cambiarLogo.setText("Cambiar Logo")

class AdminSoporteManager(QMainWindow):
    def __init__(self, ventana_login, user_role: str) -> None:
        super().__init__()
        self.ventana_login = ventana_login
        self.role = user_role

        if not self.role:
            raise TypeError("El rol de usuario no puede estar vacío.")
        self.setWindowFlag(Qt.WindowCloseButtonHint, False)
        self.setWindowIcon(QIcon(r"GUI\recursos\images\icono.ico"))
        self.setWindowTitle("GLAM MAKEUP STORE")
        self.stack = []  # Guarda las ventanas anteriores
        self.widgets_stack = QStackedWidget(self)

        # Inicializando ventanas de reporte
        self.admin_soporte = AdminSoporte(self.role)
        self.reportePanel = ReportePanel()
        self.ventas = Ventas(
            "Ventas", GD.columnas_venta_productos
        )
        self.inventarioProductos = Inventario(
            "Inventario", GD.columnas_productos
        )
        self.inventarioServicios = Inventario(
            "Inventario", GD.columnas_servicios
        )
        self.widgets_stack.addWidget(self.admin_soporte)
        self.widgets_stack.addWidget(self.reportePanel)
        self.widgets_stack.addWidget(self.ventas)
        self.widgets_stack.addWidget(self.inventarioProductos)
        self.widgets_stack.addWidget(self.inventarioServicios)

        self.admin_soporte.cerrarBtn.clicked.connect(self.volver_login)

        # Inicializando ventanas de gestión
        self.gestionPanel = GestionClientes()
        self.gestionServiciosPanel = GestionServicios()
        self.gestionVentasPanel= VentasAdmin()
        self.widgets_stack.addWidget(self.gestionPanel)
        self.widgets_stack.addWidget(self.gestionServiciosPanel)
        self.widgets_stack.addWidget(self.gestionVentasPanel)

        # Inicializando ventana de Inventario de productos
        self.principalInventarioProductosPanel = InventarioProductos()
        self.widgets_stack.addWidget(self.principalInventarioProductosPanel)

        # Inicializando ventanas de Reportes Diarios / Administracion de usuarios
        if self.role == "admin":
            self.reportesDiarios = ReportesDiarios()
            self.reportesDiariosVentas = ReporteDiarioVentas(
                "Reporte Diario de Ventas", GD.columnas_venta_productos
            )
            self.reportesDiariosProductos = ReporteDiarioProductos(
                "Reporte Diario de Productos", GD.columnas_productos
            ) 
            self.reportesDiariosCatalogo = ReporteDiarioCatalogo(
                "Reporte Diario de Catalogos", [
                    "ID",
                    "Servicio",
                    "Precio",
                ]
            )

            self.widgets_stack.addWidget(self.reportesDiarios)
            self.widgets_stack.addWidget(self.reportesDiariosVentas)
            self.widgets_stack.addWidget(self.reportesDiariosProductos)
            self.widgets_stack.addWidget(self.reportesDiariosCatalogo)
        else:
            pass ## si no es admin es porque el usuario es soporte

        # Cambiar contraseña
        self.password_change = PasswordChange()
        self.widgets_stack.addWidget(self.password_change)

        self.administrarUsuarios = AdministrarUsuarios()
        self.widgets_stack.addWidget(self.administrarUsuarios)

        # Asignando el widget central
        self.setCentralWidget(self.widgets_stack)
        self.widgets_stack.setCurrentWidget(self.admin_soporte)

        # Conexiones
        self.admin_soporte.cerrarBtn.clicked.connect(self.volver_login)
        if self.role == 'admin':
            self.password_change.volverBtn.clicked.connect(self.anterior)
            self.findChild(QPushButton, 'cambiarContraseñaBtn').clicked.connect(self.passoword_change_window)
        else:
            self.admin_soporte.pushButton_cambiarLogo.clicked.connect(self.cambiar_logo)
        self.inicializar()
        ManejarLogo().register_observer(self)

    def cambiar_logo(self):
        opciones = QFileDialog.Options()
        archivo, _ = QFileDialog.getOpenFileName(
            self,
            "Seleccionar Logo",
            "",
            "Imágenes (*.png *.jpg *.jpeg *.bmp);;Todos los archivos (*)",
            options=opciones,
        )
        if archivo:
            ManejarLogo().set_logo(archivo)

    def update_logo(self, path):
        self.admin_soporte.logo.setPixmap(QPixmap(path))

    def closeEvent(self, event):
        ManejarLogo().unregister_observer(self)
        event.accept()

    def volver_login(self):
        self.ventana_login.show()
        self.close()

    def inicializar(self):
        self.resize(1200, 800)
        self.conexiones()

        buttons = self.findChildren(QPushButton)
        for button in buttons:
            button.setCursor(QCursor(Qt.PointingHandCursor))

    def conexiones(self):
        self.admin_soporte.reportesBtn.clicked.connect(self.ventana_reportes)
        self.admin_soporte.gestionBtn.clicked.connect(self.ventana_gestionClientes)
        self.admin_soporte.catalogoBtn.clicked.connect(self.ventana_gestionServicios)
        self.admin_soporte.ventasBtn.clicked.connect(self.ventana_gestionVentas)
        self.admin_soporte.inventarioBtn.clicked.connect(self.ventana_principalInventarioProductos)
        if self.role == 'admin':
            self.findChild(QPushButton, 'reporteDiarioBtn').clicked.connect(self.ventana_reportes_diarios)
        else:
            self.findChild(QPushButton, 'administrarUsuarioBtn').clicked.connect(self.ventana_administrarUsuarios)

        self.reportePanel.volverBtn.clicked.connect(self.anterior)

        self.reportePanel.inventarioBtn.clicked.connect(self.ventana_inventario)
        self.reportePanel.ventasBtn.clicked.connect(self.ventana_ventas)
        self.ventas.volverBtn.clicked.connect(self.anterior)

        self.inventarioServicios.productosBtn.clicked.connect(self.ventana_inventarioProductos)
        self.inventarioProductos.serviciosBtn.clicked.connect(self.ventana_inventarioServicios)

        self.inventarioServicios.volverBtn.clicked.connect(self.anterior)
        self.inventarioProductos.volverBtn.clicked.connect(self.anterior)

        self.gestionPanel.atrasBtn.clicked.connect(self.anterior)
        self.gestionServiciosPanel.atrasBtn.clicked.connect(self.anterior)
        self.principalInventarioProductosPanel.atrasBtn.clicked.connect(self.anterior)

        if self.role == "admin":
            self.reportesDiarios.volverBtn.clicked.connect(self.anterior)
            self.reportesDiarios.ventasBtn.clicked.connect(self.ventana_reportes_diarios_ventas)
            self.reportesDiarios.productosBtn.clicked.connect(self.ventana_reporte_diario_productos)
            self.reportesDiarios.catalogoBtn.clicked.connect(self.ventana_reporte_diario_catalogo)

            self.reportesDiariosVentas.volverBtn.clicked.connect(self.anterior)
            self.reportesDiariosProductos.volverBtn.clicked.connect(self.anterior)
            self.reportesDiariosCatalogo.volverBtn.clicked.connect(self.anterior)
        else:
            self.administrarUsuarios.atrasBtn.clicked.connect(self.anterior)

    def ventana_reportes(self):
        self.widgets_stack.setCurrentWidget(self.reportePanel)
        self.stack.append(self.admin_soporte)

    def ventana_ventas(self):
        self.widgets_stack.setCurrentWidget(self.ventas)
        self.stack.append(self.reportePanel)

    def ventana_inventario(self):
        self.ventana_inventarioProductos()

    def ventana_inventarioServicios(self):
        self.widgets_stack.setCurrentWidget(self.inventarioServicios)
        if not self.reportePanel in self.stack:
            self.stack.append(self.reportePanel)

        self.inventarioServicios.serviciosBtn.setStyleSheet(
            "background-color: #FFFFFF;"
        )
        self.inventarioServicios.productosBtn.setStyleSheet("background-color: none;")

    def ventana_inventarioProductos(self):
        self.widgets_stack.setCurrentWidget(self.inventarioProductos)
        if not self.reportePanel in self.stack:
            self.stack.append(self.reportePanel)

        self.inventarioProductos.serviciosBtn.setStyleSheet("background-color: none;")
        self.inventarioProductos.productosBtn.setStyleSheet("background-color: #FFFFFF;")

    def ventana_gestionClientes(self):
        self.widgets_stack.setCurrentWidget(self.gestionPanel)
        self.stack.append(self.admin_soporte)

    def ventana_gestionServicios(self):
        self.widgets_stack.setCurrentWidget(self.gestionServiciosPanel)
        self.stack.append(self.admin_soporte)

    def ventana_gestionVentas(self):
        self.widgets_stack.setCurrentWidget(self.gestionVentasPanel)
        self.stack.append(self.admin_soporte)

    def ventana_principalInventarioProductos(self):
        self.widgets_stack.setCurrentWidget(self.principalInventarioProductosPanel)
        self.stack.append(self.admin_soporte)

    def ventana_reportes_diarios(self):
        self.widgets_stack.setCurrentWidget(self.reportesDiarios)
        self.stack.append(self.admin_soporte)
 
    def ventana_reportes_diarios_ventas(self):
        self.widgets_stack.setCurrentWidget(self.reportesDiariosVentas)
        self.stack.append(self.reportesDiarios)

    def ventana_reporte_diario_productos(self):
        self.widgets_stack.setCurrentWidget(self.reportesDiariosProductos)
        self.stack.append(self.reportesDiarios)

    def ventana_reporte_diario_catalogo(self):
        self.widgets_stack.setCurrentWidget(self.reportesDiariosCatalogo)
        self.stack.append(self.reportesDiarios)
        
    def ventana_administrarUsuarios(self):
        self.widgets_stack.setCurrentWidget(self.administrarUsuarios)
        self.stack.append(self.admin_soporte)

    def passoword_change_window(self):
        self.widgets_stack.setCurrentWidget(self.password_change)
        self.stack.append(self.admin_soporte)

    def anterior(self):
        anterior = self.admin_soporte
        if self.stack:
            anterior = self.stack.pop()
        self.widgets_stack.setCurrentWidget(anterior)

    def leer_estilos(self, app: QApplication, paths: List[str]) -> None:
        for path in paths:
            with open(path, "r") as style_file:
                style_line = style_file.read()
        app.setStyleSheet(style_line)
        style_file.close()

    def run(self):
        self.show()

class ManejarLogo:
    _instacia = None

    def __new__(cls):
        if cls._instacia == None:
            cls._instacia = super(ManejarLogo, cls).__new__(cls)
            cls._instacia.ruta_logo = cls._instacia.load_logo_path()
            cls._instacia.observers = []
        return cls._instacia

    @staticmethod
    def load_logo_path():
        if os.path.exists("config.json"):
            with open("config.json", "r") as file:
                config = json.load(file)
                return config.get("logo_path", r"GUI\recursos\images\logo.png")
        return r"GUI\recursos\images\logo.png"

    @staticmethod
    def save_logo_path(path):
        with open("config.json", "w") as file:
            json.dump({"logo_path": path}, file)

    def set_logo(self, path):
        self.logo_path = path
        self.save_logo_path(path)
        self.notify_observers()

    def get_logo(self):
        return self.logo_path

    def register_observer(self, observer):
        if observer not in self.observers:
            self.observers.append(observer)

    def unregister_observer(self, observer):
        if observer in self.observers:
            self.observers.remove(observer)

    def notify_observers(self):
        for observer in self.observers:
            observer.update_logo(self.logo_path)


In [ ]:
from PyQt5.uic import loadUi
import pandas as pd
from PyQt5.QtWidgets import (
    QMainWindow,
    QHeaderView,
    QTableWidgetItem,
    QMessageBox,
    QCompleter,
)
from PyQt5.QtCore import QPropertyAnimation, Qt, QStringListModel
from PyQt5 import QtCore, QtWidgets, QtGui
from API.DATA import GestionDatos
from API.Validaciones import *
from API.prueba import Cajero
from GUI.sub_ventanas.custom.utils.css import CBackground


class VentasAdmin(QMainWindow, CBackground):
    def __init__(self):
        super(VentasAdmin, self).__init__()
        loadUi(
            r"GUI\sub_ventanas\ui\Ventas.ui",
            self,
        )
        self.gestion_datos = GestionDatos()
        self.cajero = Cajero()
        self.pushButton_menu.clicked.connect(self.mover_menu)
        # Botones
        self.pushButton_actualizarPagos.clicked.connect(self.mostrar_pagos)
        self.pushButton_mostrarModificar.clicked.connect(self.mostrar_ventas)
        self.pushButton_addPago.clicked.connect(self.mostrar_formulario_addPago)
        self.pushButton_eliminarPago.clicked.connect(self.mostrar_formulario_delPago)
        self.pushButton_confirmarPago.clicked.connect(self.add_metodoPago)
        self.pushButton_delPago.clicked.connect(self.del_metodoPago)
        self.pushButton_modificarBuscar.clicked.connect(self.mostrar_formulario)
        self.pushButton_eliminarVenta.clicked.connect(self.mostrar_formularioEliminar)
        self.pushButton_ConfirmarEliminado.clicked.connect(self.ConfirmarEliminado)
        self.pushButton_CancelarEliminado.clicked.connect(self.CancelarEliminado)
        self.pushButton_guardarInfo.clicked.connect(self.guardarInfo)
        self.pushButton_fecha.clicked.connect(self.mostrar_calendario)
        self.frame_formulario.hide()
        self.frame_calendario.hide()
        self.frame_formularioEliminar.hide()
        self.frame_addPago.hide()
        self.frame_delPago.hide()
        #self.df = pd.read_excel("registros.xlsx", sheet_name="productosServicios")
        #id_ventas = self.df["ID venta"].astype(str).tolist()
        #self.modelo_datos = QStringListModel(id_ventas)
        #self.completer = QCompleter(self.modelo_datos, self)
        #self.completer.setCaseSensitivity(False)  # Ignorar mayúsculas y minúsculas
        #self.completer.setFilterMode(
        #    Qt.MatchContains
        #)  # Coincidir con cualquier parte del texto
        #self.completer.setCompletionMode(QCompleter.PopupCompletion)
        self.completer.setPopup(self.listView_buscar)
        #self.lineEdit_idVenta.setCompleter(self.completer)

        # Mas botones
        self.pushButton_pagos.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_pagos)
        )
        self.pushButton_pagos.clicked.connect(self.limpiar_campos)
        self.pushButton_ventas.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_ventas)
        )
        self.pushButton_ventas.clicked.connect(self.limpiar_campos)
    
    def limpiar_campos(self):
        None

    def mover_menu(self):
        if True:
            width = self.frame_control.width()
            normal = 0
            if width == 0:
                extender = 270
            else:
                extender = normal
            self.animacion = QPropertyAnimation(self.frame_control, b"minimumWidth")
            self.animacion.setDuration(300)
            self.animacion.setStartValue(width)
            self.animacion.setEndValue(extender)
            self.animacion.setEasingCurve(
                QtCore.QEasingCurve.InOutQuart
            )  # InQuad, InOutQuad, InCubic, InOutExpo
            self.animacion.start()



    def mostrar_pagos(self):
        None

    def mostrar_ventas(self):
        # Implement your logic to display the ventas page here
        # Similar to mostrar_pagos, it likely returns None
        return None

    def mostrar_formulario_addPago(self):
        # Implement your logic to display the add payment form here
        # Similar to mostrar_pagos, it likely returns None
        return None

    def mostrar_formulario_delPago(self):
        None

    def add_metodoPago(self):
        None

    def del_metodoPago(self):
        None

    def mostrar_formulario(self):
        None

    def mostrar_formularioEliminar(self):
        None

    def ConfirmarEliminado(self):
        None
    
    def CancelarEliminado(self):
        None
    
    def guardarInfo(self):  
        None
    def mostrar_calendario(self):
        # Implement your logic to display the calendar here
        # Similar to mostrar_pagos, it likely returns None
        return None


In [11]:
import datetime

from PyQt5 import uic

from PyQt5.QtWidgets import ( 
    QWidget, QLabel, 
    QHBoxLayout, QSpacerItem, 
    QSizePolicy, QTableWidget,
    QHeaderView, QGraphicsDropShadowEffect
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import (
    QColor, QPixmap, 
    QCursor, QIcon
)
from GUI.sub_ventanas.custom.utils.css import CustomGroupBox, CBackground
import API.DATA as GD
from GUI.sub_ventanas.custom.validaciones import CustomValidaciones
# Tipado
from typing import List, Union, Dict

GD = GD.GestionDatos()
vald = CustomValidaciones()

class ReportePorFecha(QWidget):
    def __init__(self, ref) -> None:
        super().__init__()
        uic.loadUi(
            r"GUI\sub_ventanas\ui\reportes\reportePorFecha.ui", 
            self
        )
        self.ref = ref
        self.fecha = {}

        self.setFixedSize(400, 335)
        self.setWindowIcon(QIcon(r"GUI\recursos\images\icono.ico"))

        self.setBtn.clicked.connect(self.combo_event)
        self.cancelarBtn.clicked.connect(self.close)
        self.confirmarBtn.clicked.connect(self.confirmar)

        self.pintar()

    def combo_event(self) -> None:
        comboFocus: str = self.des_hasComboBox.currentText()
        fecha_seleccionada: str = self.calendarioPorFecha.selectedDate().toPyDate()
        if comboFocus.strip().lower() == "desde":
            self.fecha["desde"] = fecha_seleccionada
            self.desdeLabel.setText(f'Desde\n{fecha_seleccionada}')
            return None

        self.fecha["hasta"] = fecha_seleccionada
        self.hastaLabel.setText(f'Hasta\n{fecha_seleccionada}')

    def confirmar(self) -> None:
        if not self.fecha or len(self.fecha) == 1:
            vald.caja_input_no_valido("Asegurese de poner ambas fechas.")
            return None
        
        fechas_compuesta = f"{self.fecha['desde']} - {self.fecha['hasta']}"
        fechas_validas = vald.validar_fechas(fechas_compuesta)

        if not fechas_validas:
            vald.caja_input_no_valido("Fechas ingresadas no validas, recuerde que la primera fecha (desde)\ndebe ser menor que la segunda(hasta).")
            return None

        self.ref.setText(fechas_compuesta)
        self.close()

    def pintar(self) -> None:
        with open(r"GUI\sub_ventanas\css\byDate.css", "r") as style_file:
            style_line = style_file.read()

        self.setStyleSheet(style_line)
        style_file.close()

class Plantilla(QWidget):
    def __init__(self, title: str, columns: List[str]) -> None:
        super().__init__()
        uic.loadUi(
            r"GUI\sub_ventanas\ui\reportes\plantillaDesigner.ui", 
            self
        )

        self.campos: List[Union[str, int, float]] = columns
        self.BD_DATA = {}

        self.titleLabel.setText(title)
        self.handle_table(data=self.BD_DATA)
        self.handle_labels()

        self.fechaBtn.clicked.connect(self.abrir_ventana_por_fecha)
        self.filtrarBtn.clicked.connect(self.filtrar)

        self.pintar()

    def handle_labels(self) -> None:
        for campo in self.campos:
            if campo.strip().lower() == 'fecha':
                continue

            contenedor = CustomGroupBox(self)
            contenedor.setObjectName("contenedorColumnas")
            contenedor.setCursor(QCursor(Qt.PointingHandCursor))
            contenedor.setContentsMargins(10, 0, 10, 0)

            decoracion = QLabel(contenedor)
            decoracion.setPixmap(QPixmap(r"GUI\recursos\images\pink_circle.png"))
            decoracion.setMaximumSize(15, 13)
            decoracion.setScaledContents(True)

            label = QLabel(contenedor)
            label.setObjectName("campo")
            label.setText(campo)
            contenedor.label = label

            vbox_layout = QHBoxLayout(contenedor)
            vbox_layout.addWidget(decoracion)
            vbox_layout.addWidget(label)

            contenedor.setLayout(vbox_layout)

            self.labelsLayout.addWidget(contenedor)

        self.labelsLayout.addItem(QSpacerItem(2, 70, QSizePolicy.Minimum, QSizePolicy.Expanding))

    def handle_table(self, data: Dict[str, Union[int, str]]) -> None:
        table: QTableWidget = self.tablaReportes
        table.setColumnCount(len(self.campos))
        table.setHorizontalHeaderLabels(self.campos)
        table.resizeColumnsToContents()
        table.verticalHeader().setDefaultSectionSize(20)

        header = table.horizontalHeader()
        header.setSectionResizeMode(QHeaderView.Stretch)

        shadow_effect = QGraphicsDropShadowEffect(self)
        shadow_effect.setBlurRadius(8)
        shadow_effect.setColor(QColor(0, 0, 0, 70))
        shadow_effect.setOffset(0, 0)
        header.setGraphicsEffect(shadow_effect)

        table.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
        table.setHorizontalScrollMode(QTableWidget.ScrollPerPixel)
        # logica para implementar los registros de la tabla

    def abrir_ventana_por_fecha(self) -> None:
        self.consultandoPor.setText("fecha")
        fecha_actual = datetime.date.today()

        self.fechas_label = self.findChild(QLabel, "rangoDeFechasLabel")
        if not self.fechas_label:
            self.fechas_label = QLabel()
            self.fechas_label.setObjectName("rangoDeFechasLabel")
            self.fechas_label.setText(f"{fecha_actual} - {fecha_actual}")
            self.cajaFiltroVerticalLayout.insertWidget(5, self.fechas_label)

        self.consulta_por_fecha = ReportePorFecha(ref=self.fechas_label)
        self.consulta_por_fecha.show()

    def filtrar(self):
        eleccion: str = self.normalizar(self.consultandoPor.text())
        campos = [campo.lower() for campo in self.campos]
        user_input: str = self.userInput.text()

        print(eleccion, user_input)

        def caja_input_no_valido():
            pass # mostrar caja

        CONRTOLADOR_DE_FILTRADO = {} # funciones para hacer query

        # return CONRTOLADOR_DE_FILTRADO[eleccion] comentado para evitar error

    def normalizar(self, cadena: str):
        cadena = cadena.strip().lower().replace(" ", "_")
        return cadena
    
    def pintar(self) -> None:
        with open(r"GUI\sub_ventanas\css\reportes_plantilla.css", "r") as style_file:
            style_line = style_file.read()

        self.setStyleSheet(style_line)
        style_file.close()

class Ventas(Plantilla):
    def __init__(self, title: str, columns: List[str | int]) -> None:
        super().__init__(title, columns)

        self.inicializar()

    def inicializar(self):
        if hasattr(self, 'navbar'):
            self.navbar.deleteLater()

class Inventario(Plantilla):
    def __init__(self, title: str, columns: List[str | int]) -> None:
        super().__init__(title, columns)

class ReportePanel(QWidget, CBackground):
    def __init__(self):
        super().__init__()
        uic.loadUi(
            r"GUI\sub_ventanas\ui\reportes\reportesDesigner.ui",
            self,
        )


In [12]:
from typing import List, Union

from PyQt5 import uic
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import (
    QWidget, QTableWidget,
    QHeaderView, QGraphicsDropShadowEffect,
    QFileDialog
)
from PyQt5.QtGui import ( QColor, QTextDocument )
from PyQt5.QtPrintSupport import QPrinter, QPrintDialog

from GUI.sub_ventanas.custom.utils.css import CBackground

class ReportesDiarios(QWidget, CBackground):
    def __init__(self) -> None:
        super().__init__()
        uic.loadUi(
            r"GUI\sub_ventanas\ui\reportesDiarios\reportesDiarios.ui", 
            self
        )

class Plantilla(QWidget):
    def __init__(self, title: str, columns: List[str]) -> None:
        super().__init__()
        uic.loadUi(
            r"GUI\sub_ventanas\ui\reportesDiarios\reportesDiariosPlantilla.ui", 
            self
        )

        self.title: str = title
        self.campos: List[Union[str, int, float]]  = columns

        self.__inicializar__()
        self.imprimirBtn.clicked.connect(self.__imprimir__)
        self.pdfBtn.clicked.connect(self.__PDF__)
        self.pintar()

    def __inicializar__(self) -> None:
        self.rDPlantillaTitle.setText(self.title)
        self.__inicializar_tabla__()

    def __inicializar_tabla__(self) -> None:
        table: QTableWidget = self.tableReporteDiario
        table.setColumnCount(len(self.campos))
        table.setHorizontalHeaderLabels(self.campos)
        table.resizeColumnsToContents()
        table.verticalHeader().setDefaultSectionSize(20)

        header = table.horizontalHeader()
        header.setSectionResizeMode(QHeaderView.Stretch)

        shadow_effect = QGraphicsDropShadowEffect(self)
        shadow_effect.setBlurRadius(8)
        shadow_effect.setColor(QColor(0, 0, 0, 70))
        shadow_effect.setOffset(0, 0)
        header.setGraphicsEffect(shadow_effect)

        table.setHorizontalScrollMode(QTableWidget.ScrollPerPixel)
        table.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOn)

    def __imprimir__(self) -> None:
        printer = QPrinter()

        dialogo = QPrintDialog(printer, self)
        if dialogo.exec_() == QPrintDialog.Accepted:
            document = QTextDocument()

            html = self.__to_html__()

            document.setHtml(html)

            document.print_(printer)

    def __to_html__(self):
        html = "<html><head></head><body><table border='1'>"

        # Añadir campos
        html += "<tr>"
        for column in range(self.tableReporteDiario.columnCount()):
            header = self.tableReporteDiario.horizontalHeaderItem(column)
            html += f"<th>{header.text()}</th>"
        html += "</tr>"

        # Añadir registros
        for row in range(self.tableReporteDiario.rowCount()):
            html += "<tr>"
            for column in range(self.tableReporteDiario.columnCount()):
                item = self.tableReporteDiario.item(row, column)
                html += f"<td>{item.text() if item else ''}</td>"
            html += "</tr>"

        html += "</table></body></html>"
        return html
    
    def __PDF__(self) -> None:
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getSaveFileName(self, "Save as PDF", "", "PDF Files (*.pdf);;All Files (*)", options=options)
        if file_path:
            printer = QPrinter(QPrinter.PdfFormat)
            printer.setOutputFileName(file_path)

            document = QTextDocument()

            html = self.table_widget_to_html()

            document.setHtml(html)

            document.print_(printer)

    def pintar(self) -> None:
        with open(r"GUI\sub_ventanas\css\reportes_diarios.css", "r") as style_file:
            style_line = style_file.read()

        self.setStyleSheet(style_line)
        style_file.close()

class ReporteDiarioVentas(Plantilla):
    def __init__(self, title: str, columns: List[str]) -> None:
        super().__init__(title, columns)

class ReporteDiarioProductos(Plantilla):
    def __init__(self, title: str, columns: List[str]) -> None:
        super().__init__(title, columns)

class ReporteDiarioCatalogo(Plantilla):
    def __init__(self, title: str, columns: List[str]) -> None:
        super().__init__(title, columns)


In [13]:
import pandas as pd
from PyQt5.uic import loadUi
from PyQt5.QtWidgets import (
    QMainWindow,
    QHeaderView,
    QTableWidgetItem,
    QCompleter,
    QMessageBox,
)
from PyQt5.QtCore import QPropertyAnimation, Qt, QStringListModel
from PyQt5 import QtCore, QtGui
from API.DATA import GestionDatos
from API.prueba import Inventario

from GUI.sub_ventanas.custom.utils.css import CBackground


class InventarioProductos(QMainWindow, CBackground):
    def __init__(self):
        super(InventarioProductos, self).__init__()
        loadUi(
            r"GUI\sub_ventanas\ui\inventario_productos\inventario_productos.ui",
            self,
        )
        self.menu_boton.clicked.connect(self.mover_menu)
        self.add_boton.clicked.connect(self.add_productos)
        self.ver_actualizar_boton.clicked.connect(self.ver_productos)
        self.pushButton_verStock.clicked.connect(self.ver_productosComprar)
        self.pushButton_verModificar.clicked.connect(self.ver_productosModificar)
        self.pushButton_verEliminar.clicked.connect(self.ver_productosEliminar)
        self.modify_buscar_boton.clicked.connect(self.mostrar_formulario)
        self.modify_guardar_boton.clicked.connect(self.modificar_productos)
        self.buy_buscar_boton.clicked.connect(self.mostrar_formulario2)
        self.buy_add_boton.clicked.connect(self.comprar_stock)
        self.inventario = Inventario()
        self.gestion_datos = GestionDatos()
        # Conexión botones barra lateral con páginas
        self.ver_productos_boton.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.ver_productos_pagina)
        )
        self.ver_productos_boton.clicked.connect(self.limpiar_campos)
        self.nuevo_producto_boton.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.nuevo_producto_pagina)
        )
        self.nuevo_producto_boton.clicked.connect(self.limpiar_campos)
        self.modificar_producto_boton.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.modificar_producto_pagina)
        )
        self.modificar_producto_boton.clicked.connect(self.limpiar_campos)
        self.descontinuar_producto_boton.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(
                self.descontinuar_producto_pagina
            )
        )
        self.descontinuar_producto_boton.clicked.connect(self.limpiar_campos)
        self.comprar_stock_boton.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.comprar_stock_pagina)
        )
        self.comprar_stock_boton.clicked.connect(self.limpiar_campos)

        # Ancho columna adaptable
        self.tabla_ver_productos.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )
        self.tableWidget_modificar.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )
        self.del_tabla_productos.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )
        self.tableWidget_comprarStock.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )
        # Llamado métodos de validación en constructor
        self.setupValidatorsCodigoBarras()
        self.setupValidatorsPrecios()
        self.setupValidatorsUnidades()
        self.frame_formularioModificar.hide()
        self.frame_formulario_comprar_stock.hide()
        self.df = pd.read_excel("registros.xlsx", sheet_name="Productos")
        self.productos = self.df["Codigo de barras"].astype(str).tolist()
        self.modelo_datos = QStringListModel(self.productos)
        self.completer = QCompleter(self.modelo_datos, self)
        self.completer.setCaseSensitivity(False)  # Ignorar mayúsculas y minúsculas
        self.completer.setFilterMode(
            Qt.MatchContains
        )  # Coincidir con cualquier parte del texto
        self.completer.setCompletionMode(QCompleter.PopupCompletion)
        self.completer.setPopup(self.listView_comprarStock)
        self.completer.setPopup(self.listView_modificar)
        self.completer.setPopup(self.listView_eliminar)
        self.modify_buscar_producto_lineEdit.setCompleter(self.completer)
        self.del_buscar_producto_lineEdit.setCompleter(self.completer)
        self.buy_buscar_producto_lineEdit.setCompleter(self.completer)

    # Método para limpiar los campos cada que se cambiar de página
    def limpiar_campos(self):
        # Labels añadir_producto
        self.add_referencia_lineEdit.clear()
        self.add_marca_lineEdit.clear()
        self.add_precio_adquisicion_lineEdit.clear()
        self.add_precio_ventas_lineEdit.clear()
        self.add_unidades_actuales_lineEdit.clear()
        self.add_codigoBarras_lineEdit.clear()
        self.modify_buscar_producto_lineEdit.clear()
        self.modify_marca_lineEdit.clear()
        self.modify_precio_adquisicion_lineEdit.clear()
        self.modify_precio_venta_lineEdit.clear()
        self.del_buscar_producto_lineEdit.clear()
        self.buy_buscar_producto_lineEdit.clear()
        self.buy_cantidad_ingresar_lineEdit.clear()
        self.aviso_add_label.setText("")
        self.aviso_modify_label.setText("")
        self.del_aviso_label.setText("")
        self.buy_aviso_label.setText("")

    # Método para validar la longitud del código de barras
    def setupValidatorsCodigoBarras(self):
        cbarras = QtGui.QRegularExpressionValidator(
            QtCore.QRegularExpression(r"\d{0,13}")
        )
        self.add_codigoBarras_lineEdit.setValidator(cbarras)
        self.modify_buscar_producto_lineEdit.setValidator(cbarras)
        self.del_buscar_producto_lineEdit.setValidator(cbarras)
        self.buy_buscar_producto_lineEdit.setValidator(cbarras)

    # Método para definir los precios
    def setupValidatorsPrecios(self):
        # Límite recomendado para precios (hasta 99999999.99)
        validacion_precios = QtGui.QRegularExpressionValidator(
            QtCore.QRegularExpression(r"\d{1,12}(\.\d{1,2})?")
        )
        self.add_precio_adquisicion_lineEdit.setValidator(validacion_precios)
        self.add_precio_ventas_lineEdit.setValidator(validacion_precios)
        self.modify_precio_adquisicion_lineEdit.setValidator(validacion_precios)
        self.modify_precio_venta_lineEdit.setValidator(validacion_precios)

    # Método para definir la cantidad de unidades (máximo 99.999)
    def setupValidatorsUnidades(self):
        validacion_unidades = QtGui.QRegularExpressionValidator(
            QtCore.QRegularExpression(r"\d{0,7}")
        )
        self.add_unidades_actuales_lineEdit.setValidator(validacion_unidades)
        self.buy_cantidad_ingresar_lineEdit.setValidator(validacion_unidades)

    # Método que permite mover la barra de menú
    def mover_menu(self):
        if True:
            width = self.frame_control.width()
            normal = 0
            if width == 0:
                extender = 270
            else:
                extender = normal
            self.animacion = QPropertyAnimation(self.frame_control, b"minimumWidth")
            self.animacion.setDuration(300)
            self.animacion.setStartValue(width)
            self.animacion.setEndValue(extender)
            self.animacion.setEasingCurve(
                QtCore.QEasingCurve.InOutQuart
            )  # InQuad, InOutQuad, InCubic, InOutExpo
            self.animacion.start()

    def showErrorMessage(self, message):
        msg_box = QMessageBox()
        msg_box.setIcon(QMessageBox.Warning)
        msg_box.setText(message)
        msg_box.setWindowTitle("Error de autenticación")
        msg_box.exec_()

    def show_success_dialog(self, message):
        msg_box = QMessageBox()
        msg_box.setIcon(QMessageBox.Information)
        msg_box.setText(message)
        msg_box.setWindowTitle("Éxito")
        msg_box.exec_()

    # Método para definir base de datos ...
    # self.tabla_ver_productos.setRowCount(0)

    def add_productos(self):
        referencia = self.add_referencia_lineEdit.text()
        marca = self.add_marca_lineEdit.text()
        precio_adquisicion = self.add_precio_adquisicion_lineEdit.text()
        precio_venta = self.add_precio_ventas_lineEdit.text()
        unidades_actuales = self.add_unidades_actuales_lineEdit.text()
        codigo_barras = self.add_codigoBarras_lineEdit.text()
        if precio_venta and precio_venta and unidades_actuales != "":
            if self.inventario.crear_productos(
                referencia,
                precio_adquisicion,
                precio_venta,
                codigo_barras,
                marca,
                unidades_actuales,
            ):
                self.ver_productos()
                self.show_success_dialog("Producto agregado correctamente")
                self.aviso_add_label.setText("Producto agregado correctamente")
                self.limpiar_campos()
            else:
                self.showErrorMessage(
                    "Producto no agregado. Por favor, verifica la información."
                )
                self.aviso_add_label.setText(
                    "Producto no agregado. Por favor, verifica la información."
                )
        else:
            self.showErrorMessage(
                    "Algun campo esta vacío. Por favor, verifica la información."
                )
            self.aviso_add_label.setText(
                "Algun campo esta vacío. Por favor, verifica la información."
            )

    def ver_productos(self):
        self.tabla_ver_productos.setRowCount(0)
        for i, row in self.df.iterrows():
            self.tabla_ver_productos.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.tabla_ver_productos.setItem(i, j, QTableWidgetItem(str(value)))

    def ver_productosModificar(self):
        self.tableWidget_modificar.setRowCount(0)
        for i, row in self.gestion_datos.productos.iterrows():
            self.tableWidget_modificar.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.tableWidget_modificar.setItem(i, j, QTableWidgetItem(str(value)))

    def ver_productosEliminar(self):
        self.del_tabla_productos.setRowCount(0)
        for i, row in self.gestion_datos.productos.iterrows():
            self.del_tabla_productos.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.del_tabla_productos.setItem(i, j, QTableWidgetItem(str(value)))

    def ver_productosComprar(self):
        self.tableWidget_comprarStock.setRowCount(0)
        for i, row in self.gestion_datos.productos.iterrows():
            self.tableWidget_comprarStock.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.tableWidget_comprarStock.setItem(
                    i, j, QTableWidgetItem(str(value))
                )

    def mostrar_formulario(self):
        if self.modify_buscar_producto_lineEdit.text() != "":
            # print(self.productos)
            codigo_barras = self.gestion_datos.buscar_producto(
                self.modify_buscar_producto_lineEdit.text()
            )
            # print(codigo_barras)
            if codigo_barras:
                self.frame_formularioModificar.show()
            else:
                # print(codigo_barras)
                self.showErrorMessage(
                    "Producto Inexistente. Por favor, verifica la información."
                )
                self.aviso_modify_label.setText(
                    "Producto Inexistente. Por favor, verifica la información."
                )
        else:
            self.showErrorMessage("Campo vacío. Por favor, verifica la información.")
            self.aviso_modify_label.setText(
                "Campo vacío. Por favor, verifica la información."
            )

    def modificar_productos(self):
        if (
            self.modify_precio_adquisicion_lineEdit.text()
            and self.modify_precio_venta_lineEdit.text() != ""
        ):
            codigo = int(self.modify_buscar_producto_lineEdit.text())
            if codigo in self.gestion_datos.productos["Codigo de barras"].values:
                datos_producto = self.gestion_datos.productos[
                    self.gestion_datos.productos["Codigo de barras"] == codigo
                ]
            elif codigo in self.gestion_datos.productos["Codigo de barras"].values:
                datos_producto = self.gestion_datos.productos[
                    self.gestion_datos.productos["Codigo de barras"] == codigo
                ]
            marca = self.modify_marca_lineEdit.text()
            precio_a = float(self.modify_precio_adquisicion_lineEdit.text())
            precio_v = float(self.modify_precio_venta_lineEdit.text())
            if not datos_producto.empty:
                self.inventario.modificar_producto(
                    marca, precio_a, precio_v, codigo, datos_producto
                )
                self.ver_productos()  # Actualizar la tabla de productos
                self.show_success_dialog("Cliente modificado correctamente")
                self.aviso_modify_label.setText("Cliente modificado correctamente")
                self.limpiar_campos()
                self.frame_formularioModificar.hide()
                # print(datos_producto)
        else:
            self.showErrorMessage("Campos vacíos. Porfavor, verifique la información")
            self.aviso_modify_label.setText(
                "Campos vacíos. Porfavor, verifique la información"
            )

    def descontinuar_producto(self):  # Falta terminar
        codigo = self.del_buscar_producto_lineEdit.text()
        if codigo in self.gestion_datos.productos["Codigo de barras"].values:
            self.inventario.descontinuar_producto(codigo)
        elif int(codigo) in self.gestion_datos.productos["Codigo de barras"].values:
            codigo = int(codigo)
            self.inventario.descontinuar_producto(codigo)

    def verificar_existencia_stock(self):
        if not self.buy_buscar_producto_lineEdit.text():
            codigo = self.buy_buscar_producto_lineEdit.text()
            if (
                codigo in self.gestion_datos.productos["Codigo de barras"].values
                or codigo in self.gestion_datos.productos["Codigo de barras"].values
            ):
                return True
            else:
                return False
        else:
            self.showErrorMessage("Campo vacío, por favor ingrese la información.")

    def mostrar_formulario2(self):
        if not self.buy_cantidad_ingresar_lineEdit.text():
            codigo_barras = self.verificar_existencia_stock()
            if codigo_barras == True:
                self.frame_formulario_comprar_stock.show()
            else:
                self.showErrorMessage(
                    "Producto Inexistente. Por favor, verifica la información."
                )
                self.aviso_modify_label.setText(
                    "Producto Inexistente. Por favor, verifica la información."
                )
        else:
            self.showErrorMessage("Campo vacío. Por favor, verifica la información.")
            self.buy_aviso_label.setText(
                "Campo vacío. Por favor, verifica la información."
            )

    def comprar_stock(self):
        if self.verificar_existencia_stock():
            codigo = self.buy_buscar_producto_lineEdit.text()
            stock = self.buy_cantidad_ingresar_lineEdit.text()
            if int(codigo) in self.gestion_datos.productos["Codigo de barras"].values:
                codigo = int(codigo)
            self.inventario.comprar_stock(codigo, int(stock))
            self.ver_productosComprar()


In [14]:
from PyQt5.uic import loadUi
import pandas as pd
from PyQt5.QtWidgets import (
    QMainWindow,
    QHeaderView,
    QTableWidgetItem,
    QMessageBox,
    QCompleter,
)
from PyQt5.QtCore import QPropertyAnimation, Qt, QStringListModel
from PyQt5 import QtCore, QtWidgets, QtGui
from API.DATA import GestionDatos
from API.Validaciones import *
from API.prueba import Cajero

from GUI.sub_ventanas.custom.utils.css import CBackground


class GestionClientes(QMainWindow, CBackground):
    def __init__(self):
        super(GestionClientes, self).__init__()
        loadUi(
            r"GUI\sub_ventanas\ui\gestion_clientes\GestionClientes.ui",
            self,
        )
        self.gestion_datos = GestionDatos()
        self.cajero = Cajero()
        self.mostrar_clientes()
        self.mostrar_clientesModificar()
        self.mostrar_clientesEliminar()
        self.pushButton_menu.clicked.connect(self.mover_menu)
        # Botones
        self.pushButton_actualizar.clicked.connect(self.mostrar_clientes)
        self.pushButton_mostrarModificar.clicked.connect(self.mostrar_clientesModificar)
        self.pushButton_mostrarEliminar.clicked.connect(self.mostrar_clientesEliminar)
        self.pushButton_add.clicked.connect(self.registrar_cliente)
        self.pushButton_guardarInfo.clicked.connect(self.modificar_cliente)
        self.pushButton_eliminar.clicked.connect(self.eliminar_cliente)
        self.pushButton_modificar.clicked.connect(self.mostrar_formulario)

        self.gripSize = 10
        self.grip = QtWidgets.QSizeGrip(self)
        self.grip.resize(self.gripSize, self.gripSize)

        # Mas botones
        self.pushButton_verClientes.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_consulta)
        )
        self.pushButton_verClientes.clicked.connect(self.limpiar_campos)
        self.pushButton_addCliente.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_add)
        )
        self.pushButton_addCliente.clicked.connect(self.limpiar_campos)
        self.pushButton_modificarCliente.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_modificar)
        )
        self.pushButton_modificarCliente.clicked.connect(self.limpiar_campos)
        self.pushButton_eliminarCliente.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_eliminar)
        )
        self.pushButton_eliminarCliente.clicked.connect(self.limpiar_campos)

        # Ancho columna adaptable
        self.tabla_verClientes.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )

        self.tableaWidget_Eliminar.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )
        self.tableWidget_modificar.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )
        self.setupValidatorsCedula()
        self.setupValidatorsTelefono()
        self.frame_formulario.hide()
        df = pd.read_excel("registros.xlsx", sheet_name="Clientes")
        cedulas = df["Cedula"].astype(str).tolist()
        self.modelo_datos = QStringListModel(cedulas)
        self.completer = QCompleter(self.modelo_datos, self)
        self.completer.setCaseSensitivity(False)  # Ignorar mayúsculas y minúsculas
        self.completer.setFilterMode(
            Qt.MatchContains
        )  # Coincidir con cualquier parte del texto
        self.completer.setCompletionMode(QCompleter.PopupCompletion)
        self.completer.setPopup(self.listView_buscar)
        self.completer.setPopup(self.listView_eliminar)
        self.lineEdit_modificar.setCompleter(self.completer)
        self.lineEdit_buscarEliminar.setCompleter(self.completer)

    def setupValidatorsCedula(self):
        validacion_numero = QtGui.QRegularExpressionValidator(
            QtCore.QRegularExpression(r"\d{9,12}")
        )
        self.lineEdit_addCedula.setValidator(validacion_numero)
        self.lineEdit_nuevaCedula.setValidator(validacion_numero)
        self.lineEdit_buscarEliminar.setValidator(validacion_numero)
        self.lineEdit_modificar.setValidator(validacion_numero)

    def setupValidatorsTelefono(self):
        validacion_numero = QtGui.QRegularExpressionValidator(
            QtCore.QRegularExpression(r"\d{0,16}")
        )
        self.lineEdit_addTelefono.setValidator(validacion_numero)

    def limpiar_campos(self):
        self.lineEdit_addTelefono.clear()
        self.lineEdit_addCedula.clear()
        self.lineEdit_addNombre.clear()
        self.lineEdit_buscarEliminar.clear()
        self.lineEdit_nuevaCedula.clear()
        self.lineEdit_nuevoNombre.clear()
        self.lineEdit_nuevoTelefono.clear()

    def mover_menu(self):
        if True:
            width = self.frame_control.width()
            normal = 0
            if width == 0:
                extender = 270
            else:
                extender = normal
            self.animacion = QPropertyAnimation(self.frame_control, b"minimumWidth")
            self.animacion.setDuration(300)
            self.animacion.setStartValue(width)
            self.animacion.setEndValue(extender)
            self.animacion.setEasingCurve(
                QtCore.QEasingCurve.InOutQuart
            )  # InQuad, InOutQuad, InCubic, InOutExpo
            self.animacion.start()

    def showErrorMessage(self, message):
        msg_box = QMessageBox()
        msg_box.setIcon(QMessageBox.Warning)
        msg_box.setText(message)
        msg_box.setWindowTitle("Error")
        msg_box.exec_()

    def show_success_dialog(self, message):
        msg_box = QMessageBox()
        msg_box.setIcon(QMessageBox.Information)
        msg_box.setText(message)
        msg_box.setWindowTitle("Éxito")
        msg_box.exec_()

    # Acá se configura la base de datos
    def mostrar_clientes(self):
        self.tabla_verClientes.setRowCount(0)
        for i, row in self.gestion_datos.clientes.iterrows():
            self.tabla_verClientes.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.tabla_verClientes.setItem(i, j, QTableWidgetItem(str(value)))

    def mostrar_clientesModificar(self):
        self.tableWidget_modificar.setRowCount(0)
        for i, row in self.gestion_datos.clientes.iterrows():
            self.tableWidget_modificar.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.tableWidget_modificar.setItem(i, j, QTableWidgetItem(str(value)))

    def mostrar_clientesEliminar(self):
        self.tableWidget_Eliminar.setRowCount(0)
        for i, row in self.gestion_datos.clientes.iterrows():
            self.tableWidget_Eliminar.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.tableWidget_Eliminar.setItem(i, j, QTableWidgetItem(str(value)))

    def registrar_cliente(self):
        cedula = self.lineEdit_addCedula.text()
        nombre = self.lineEdit_addNombre.text()
        telefono = self.lineEdit_addTelefono.text()
        if (
            validar_Cedula(cedula)
            and validacion_Telefono(telefono)
            and validar_NombreCom(nombre)
            and cedula not in self.gestion_datos.clientes["Cedula"].values
        ):
            self.gestion_datos.agregar_cliente(cedula, nombre, telefono)
            self.mostrar_clientes()
            self.show_success_dialog("Cliente registrado con éxito.")
            self.aviso_add.setText("Cliente registrado con éxito.")
            self.limpiar_campos()
            self.frame_formulario.hide()
        else:
            self.showErrorMessage(
                "Error en los datos ingresados. Por favor, verifica la información."
            )
            self.aviso_add.setText(
                "Error en los datos ingresados. Por favor, verifica la información."
            )

    def validar_existencia(self):
        # Los campos para validar son:
        if self.lineEdit_modificar.text():
            cedulaBuscarCliente = int(self.lineEdit_modificar.text())
            if (
                str(cedulaBuscarCliente)
                in str(self.gestion_datos.clientes["Cedula"].values)
                or cedulaBuscarCliente in self.gestion_datos.clientes["Cedula"].values
            ):
                return True
            else:
                return False
        else:
            self.aviso_modificar.setText(
                "Campo Vacio. Por favor ingrese la información correspondiente."
            )

    def mostrar_formulario(self):
        cedula = self.validar_existencia()
        if cedula:
            self.frame_formulario.show()
            self.aviso_modificar.setText("")
        else:
            self.showErrorMessage(
                "Cédula Inexistente. Por favor, verifica la información."
            )
            self.aviso_modificar.setText(
                "Cédula Inexistente. Por favor, verifica la información."
            )

    def modificar_cliente(self):
        cedulaBuscarCliente = int(self.lineEdit_modificar.text())
        if str(cedulaBuscarCliente) in self.gestion_datos.clientes["Cedula"].values:
            datos_cliente = self.gestion_datos.clientes[
                self.gestion_datos.clientes["Cedula"] == str(cedulaBuscarCliente)
            ]
        elif cedulaBuscarCliente in self.gestion_datos.clientes["Cedula"].values:
            datos_cliente = self.gestion_datos.clientes[
                self.gestion_datos.clientes["Cedula"] == cedulaBuscarCliente
            ]
        print(datos_cliente)
        nuevaCedula = self.lineEdit_nuevaCedula.text()
        nuevoNombre = self.lineEdit_nuevoNombre.text()
        nuevoTelefono = self.lineEdit_nuevoTelefono.text()
        if not datos_cliente.empty:
            if self.cajero.modificar_cliente(
                nuevaCedula,
                nuevoNombre,
                nuevoTelefono,
                cedulaBuscarCliente,
                datos_cliente,
            ):
                self.mostrar_clientes()  # Actualizar la tabla de clientes
                self.show_success_dialog("Cliente modificado correctamente")
                self.aviso_modificar.setText("Cliente modificado correctamente")
                self.limpiar_campos()
                self.frame_formulario.hide()
            else:
                self.showErrorMessage(
                    "Error en los datos ingresados. Por favor, verifica la información."
                )
                self.aviso_add.setText(
                    "Error en los datos ingresados. Por favor, verifica la información."
                )
        else:
            self.showErrorMessage(
                "Error en los datos ingresados. Por favor, verifica la información."
            )
            self.aviso_eliminar.setText(
                "Error en los datos ingresados. Por favor, verifica la información."
            )

    def eliminar_cliente(self):
        if not self.lineEdit_buscarEliminar.text():
            self.aviso_eliminar.setText("Cédula Inexistente. Por favor, verifica la información.")
            return self.showErrorMessage(
                "Cédula Inexistente. Por favor, verifica la información."
            )
        cedula = int(self.lineEdit_buscarEliminar.text())
        eliminado = self.gestion_datos.eliminar_clientes(cedula)
        if cedula:
            if (
                eliminado
            ):  # No elimina nada, solo añadí el if para usar los valores de retorno del metodo y mostrar las ventanas emergentes
                self.mostrar_clientes()
                self.show_success_dialog("Cliente eliminado correctamente")
                self.aviso_eliminar.setText("Cliente eliminado correctamente")
                self.limpiar_campos()
            else:
                self.showErrorMessage(
                    "Error en los datos ingresados. Por favor, verifica la información."
                )
                self.aviso_eliminar.setText(
                    "Error en los datos ingresados. Por favor, verifica la información."
                )


In [15]:
from PyQt5.uic import loadUi
import pandas as pd
from PyQt5.QtWidgets import (
    QMainWindow,
    QHeaderView,
    QTableWidgetItem,
    QMessageBox,
    QCompleter,
)
from PyQt5.QtCore import QPropertyAnimation, Qt, QStringListModel
from PyQt5 import QtCore, QtWidgets, QtGui
from API.DATA import GestionDatos
from API.Validaciones import *
from API.prueba import Inventario

from GUI.sub_ventanas.custom.utils.css import CBackground


class GestionServicios(QMainWindow, CBackground):
    def __init__(self):
        super(GestionServicios, self).__init__()
        loadUi(
            r"GUI\sub_ventanas\ui\GestionServicios.ui",
            self,
        )
        self.gestion_datos = GestionDatos()
        self.inventario = Inventario()

        self.pushButton_menu.clicked.connect(self.mover_menu)
        # Botones
        self.pushButton_actualizar.clicked.connect(self.mostrar_servicios)
        self.pushButton_mostrarModificar.clicked.connect(self.mostrar_servicioModificar)
        self.pushButton_mostrarEliminar.clicked.connect(self.mostrar_servicioEliminar)
        self.pushButton_add.clicked.connect(self.registrar_servicio)
        self.pushButton_guardarInfo.clicked.connect(self.modificar_servicio)
        self.pushButton_eliminar.clicked.connect(self.eliminar_servicio)
        self.pushButton_modificar.clicked.connect(self.mostrar_formulario)

        self.gripSize = 10
        self.grip = QtWidgets.QSizeGrip(self)
        self.grip.resize(self.gripSize, self.gripSize)

        # Mas botones
        self.pushButton_verServicios.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_consulta)
        )
        self.pushButton_verServicios.clicked.connect(self.limpiar_campos)
        self.pushButton_addServicio.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_add)
        )
        self.pushButton_addServicio.clicked.connect(self.limpiar_campos)
        self.pushButton_modificarServicio.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_modificar)
        )
        self.pushButton_modificarServicio.clicked.connect(self.limpiar_campos)
        self.pushButton_eliminarServicio.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_eliminar)
        )
        self.pushButton_eliminarServicio.clicked.connect(self.limpiar_campos)

        # Ancho columna adaptable
        self.tabla_verServicios.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )
        self.tableWidget_eliminarServicio.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )
        self.tableWidget_modificar.horizontalHeader().setSectionResizeMode(
            QHeaderView.Stretch
        )
        self.setupValidatorsId()
        self.setupValidatorsPrecio()
        self.frame_formulario.hide()
        df = pd.read_excel("registros.xlsx", sheet_name="Servicios")
        servicios = df["ID servicio"].astype(str).tolist()
        self.modelo_datos = QStringListModel(servicios)
        self.completer = QCompleter(self.modelo_datos, self)
        self.completer.setCaseSensitivity(False)  # Ignorar mayúsculas y minúsculas
        self.completer.setFilterMode(
            Qt.MatchContains
        )  # Coincidir con cualquier parte del texto
        self.completer.setCompletionMode(QCompleter.PopupCompletion)
        self.completer.setPopup(self.listView_buscar)
        self.completer.setPopup(self.listView_eliminar)
        self.lineEdit_modificar.setCompleter(self.completer)
        self.lineEdit_buscarEliminar.setCompleter(self.completer)

    def resizeEvent(self, event):
        rect = self.rect()
        self.grip.move(rect.right() - self.gripSize, rect.bottom() - self.gripSize)

    def setupValidatorsId(self):
        validacion_numero = QtGui.QRegularExpressionValidator(
            QtCore.QRegularExpression(r"\d{12}")
        )
        self.lineEdit_idServicio.setValidator(validacion_numero)
        self.lineEdit_nuevoId.setValidator(validacion_numero)
        self.lineEdit_modificar.setValidator(validacion_numero)
        self.lineEdit_buscarEliminar.setValidator(validacion_numero)

    def setupValidatorsPrecio(self):
        validacion_numero = QtGui.QRegularExpressionValidator(
            QtCore.QRegularExpression(r"\d{16}")
        )
        self.lineEdit_addPrecio.setValidator(validacion_numero)
        self.lineEdit_nuevoPrecio.setValidator(validacion_numero)
        self.lineEdit_modificar.setValidator(validacion_numero)
        self.lineEdit_buscarEliminar.setValidator(validacion_numero)

    def limpiar_campos(self):
        self.lineEdit_nuevoId.clear()
        self.lineEdit_idServicio.clear()
        self.lineEdit_addPrecio.clear()
        self.lineEdit_addServicio.clear()
        self.lineEdit_buscarEliminar.clear()
        self.lineEdit_nuevoServicio.clear()
        self.lineEdit_nuevoPrecio.clear()
        self.lineEdit_modificar.clear()

    def mover_menu(self):
        if True:
            width = self.frame_control.width()
            normal = 0
            if width == 0:
                extender = 270
            else:
                extender = normal
            self.animacion = QPropertyAnimation(self.frame_control, b"minimumWidth")
            self.animacion.setDuration(300)
            self.animacion.setStartValue(width)
            self.animacion.setEndValue(extender)
            self.animacion.setEasingCurve(
                QtCore.QEasingCurve.InOutQuart
            )  # InQuad, InOutQuad, InCubic, InOutExpo
            self.animacion.start()

    # Acá se configura la base de datos
    def mostrar_servicios(self):
        self.tabla_verServicios.setRowCount(0)
        for i, row in self.gestion_datos.servicios.iterrows():
            self.tabla_verServicios.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.tabla_verServicios.setItem(i, j, QTableWidgetItem(str(value)))

    def mostrar_servicioModificar(self):
        self.tableWidget_modificar.setRowCount(0)
        for i, row in self.gestion_datos.servicios.iterrows():
            self.tableWidget_modificar.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.tableWidget_modificar.setItem(i, j, QTableWidgetItem(str(value)))

    def mostrar_servicioEliminar(self):
        self.tableWidget_eliminarServicio.setRowCount(0)
        for i, row in self.gestion_datos.servicios.iterrows():
            self.tableWidget_eliminarServicio.insertRow(i)
            for j, (colname, value) in enumerate(row.items()):
                self.tableWidget_eliminarServicio.setItem(
                    i, j, QTableWidgetItem(str(value))
                )

    def showErrorMessage(self, message):
        msg_box = QMessageBox()
        msg_box.setIcon(QMessageBox.Warning)
        msg_box.setText(message)
        msg_box.setWindowTitle("Error de autenticación")
        msg_box.exec_()

    def show_success_dialog(self, message):
        msg_box = QMessageBox()
        msg_box.setIcon(QMessageBox.Information)
        msg_box.setText(message)
        msg_box.setWindowTitle("Éxito")
        msg_box.exec_()

    def validar_existencia(self):
        # Los campos para validar son:
        if self.lineEdit_modificar.text():
            idBuscarServicio = self.lineEdit_modificar.text()
            if (
                idBuscarServicio in self.gestion_datos.servicios["ID servicio"].values
                or int(idBuscarServicio)
                in self.gestion_datos.servicios["ID servicio"].values
            ):
                return True
            else:
                return False
        else:
            self.aviso_modificar.setText(
                "Campo Vacio. Por favor ingrese la información correspondiente."
            )

    def mostrar_formulario(self, servicio):
        servicio = self.validar_existencia()
        if servicio:
            self.frame_formulario.show()
        else:
            self.showErrorMessage(
                "Error en los datos ingresados. Por favor, verifica la información."
            )
            self.aviso_modificar.setText(
                "Error en los datos ingresados. Por favor, verifica la información."
            )

    def modificar_servicio(self):
        id_servicio = self.lineEdit_nuevoId.text()
        nombre_servicio = self.lineEdit_nuevoServicio.text()
        precio = int(self.lineEdit_nuevoPrecio.text())
        id_original = self.lineEdit_modificar.text()

        if int(id_original) in self.gestion_datos.servicios["ID servicio"].values:
            datos_servicio = self.gestion_datos.servicios[
                self.gestion_datos.servicios["ID servicio"] == int(id_original)
            ]
            id_original = int(id_original)
        elif id_original in self.gestion_datos.servicios["ID servicio"].values:
            datos_servicio = self.gestion_datos.servicios[
                self.gestion_datos.servicios["ID servicio"] == id_original
            ]
        if not datos_servicio.empty:
            self.inventario.modificar_servicio(
                id_servicio, nombre_servicio, precio, id_original, datos_servicio
            )
            self.mostrar_servicios()  # Actualizar la tabla de servicios

    def eliminar_servicio(self):
        id_servicio = self.lineEdit_buscarEliminar.text()
        self.gestion_datos.eliminar_servicio(id_servicio)
        self.mostrar_servicios()  # Actualizar la tabla de clientes

    def registrar_servicio(self):
        if (
            self.lineEdit_idServicio.text()
            and self.lineEdit_addServicio.text()
            and self.lineEdit_addPrecio.text() != ""
        ):
            id = int(self.lineEdit_idServicio.text())
            nombre = self.lineEdit_addServicio.text()
            precio = int(self.lineEdit_addPrecio.text())
            if self.inventario.crear_servicio(id, nombre, precio):
                self.mostrar_servicios()
                self.label_aviso.setText("Servicio registrado con éxito")
                self.show_success_dialog("Servicio registrado con éxito.")
                self.limpiar_campos()
            else:
                self.showErrorMessage(
                    "Error en los datos ingresados. Por favor, verifica la información."
                )
                self.label_aviso.setText(
                    "Error en los datos ingresados. Por favor, verifica la información."
                )
        else:
            self.label_aviso.setText(
                "Campos vacíos. Por favor, verifica la información."
            )
            self.showErrorMessage("Campos vacíos. Por favor, verifica la información.")

    def descontinuar_servicio(self):
        if not self.lineEdit_buscarEliminar:
            id_buscar = self.lineEdit_buscarEliminar.text()
            if id_buscar in self.gestion_datos.servicios["ID servicios"].values:
                self.inventario.eliminar_servicio(id_buscar)
            elif int(id_buscar) in self.gestion_datos.servicios["ID servicios"].values:
                id_buscar = int(id_buscar)
                self.inventario.eliminar_servicio(id_buscar)
                self.aviso_eliminar.setText("Cliente Eliminado.")
        else:
            self.aviso_eliminar.setText(
                "Campos vacíos. Por favor, verifica la información."
            )
            self.showErrorMessage("Campos vacíos. Por favor, verifica la información.")


In [ ]:
from PyQt5.uic import loadUi
import pandas as pd
from PyQt5.QtWidgets import (
    QMainWindow,
    QHeaderView,
    QTableWidgetItem,
    QMessageBox,
    QCompleter,
)
from PyQt5.QtCore import QPropertyAnimation, Qt, QStringListModel
from PyQt5 import QtCore, QtWidgets, QtGui
from API.DATA import GestionDatos
from API.Validaciones import *
from API.prueba import Cajero
from GUI.sub_ventanas.custom.utils.css import CBackground


class VentasAdmin(QMainWindow, CBackground):
    def __init__(self):
        super(VentasAdmin, self).__init__()
        loadUi(
            r"GUI\sub_ventanas\ui\Ventas.ui",
            self,
        )
        self.gestion_datos = GestionDatos()
        self.cajero = Cajero()
        self.pushButton_menu.clicked.connect(self.mover_menu)
        # Botones
        self.pushButton_actualizarPagos.clicked.connect(self.mostrar_pagos)
        self.pushButton_mostrarModificar.clicked.connect(self.mostrar_ventas)
        self.pushButton_addPago.clicked.connect(self.mostrar_formulario_addPago)
        self.pushButton_eliminarPago.clicked.connect(self.mostrar_formulario_delPago)
        self.pushButton_confirmarPago.clicked.connect(self.add_metodoPago)
        self.pushButton_delPago.clicked.connect(self.del_metodoPago)
        self.pushButton_modificarBuscar.clicked.connect(self.mostrar_formulario)
        self.pushButton_eliminarVenta.clicked.connect(self.mostrar_formularioEliminar)
        self.pushButton_ConfirmarEliminado.clicked.connect(self.ConfirmarEliminado)
        self.pushButton_CancelarEliminado.clicked.connect(self.CancelarEliminado)
        self.pushButton_guardarInfo.clicked.connect(self.guardarInfo)
        self.pushButton_fecha.clicked.connect(self.mostrar_calendario)
        self.frame_formulario.hide()
        self.frame_calendario.hide()
        self.frame_formularioEliminar.hide()
        self.frame_addPago.hide()
        self.frame_delPago.hide()
        #self.df = pd.read_excel("registros.xlsx", sheet_name="productosServicios")
        #id_ventas = self.df["ID venta"].astype(str).tolist()
        #self.modelo_datos = QStringListModel(id_ventas)
        #self.completer = QCompleter(self.modelo_datos, self)
        #self.completer.setCaseSensitivity(False)  # Ignorar mayúsculas y minúsculas
        #self.completer.setFilterMode(
        #    Qt.MatchContains
        #)  # Coincidir con cualquier parte del texto
        #self.completer.setCompletionMode(QCompleter.PopupCompletion)
        self.completer.setPopup(self.listView_buscar)
        #self.lineEdit_idVenta.setCompleter(self.completer)

        # Mas botones
        self.pushButton_pagos.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_pagos)
        )
        self.pushButton_pagos.clicked.connect(self.limpiar_campos)
        self.pushButton_ventas.clicked.connect(
            lambda: self.stackedWidget.setCurrentWidget(self.pagina_ventas)
        )
        self.pushButton_ventas.clicked.connect(self.limpiar_campos)
    
    def limpiar_campos(self):
        None

    def mover_menu(self):
        if True:
            width = self.frame_control.width()
            normal = 0
            if width == 0:
                extender = 270
            else:
                extender = normal
            self.animacion = QPropertyAnimation(self.frame_control, b"minimumWidth")
            self.animacion.setDuration(300)
            self.animacion.setStartValue(width)
            self.animacion.setEndValue(extender)
            self.animacion.setEasingCurve(
                QtCore.QEasingCurve.InOutQuart
            )  # InQuad, InOutQuad, InCubic, InOutExpo
            self.animacion.start()



    def mostrar_pagos(self):
        None

    def mostrar_ventas(self):
        # Implement your logic to display the ventas page here
        # Similar to mostrar_pagos, it likely returns None
        return None

    def mostrar_formulario_addPago(self):
        # Implement your logic to display the add payment form here
        # Similar to mostrar_pagos, it likely returns None
        return None

    def mostrar_formulario_delPago(self):
        None

    def add_metodoPago(self):
        None

    def del_metodoPago(self):
        None

    def mostrar_formulario(self):
        None

    def mostrar_formularioEliminar(self):
        None

    def ConfirmarEliminado(self):
        None
    
    def CancelarEliminado(self):
        None
    
    def guardarInfo(self):  
        None
    def mostrar_calendario(self):
        # Implement your logic to display the calendar here
        # Similar to mostrar_pagos, it likely returns None
        return None


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication
from GUI.ventanas.login import Login

class Main:
    def __init__(self):
        self.app = QApplication(sys.argv)
        self.login_window = Login(self.app) 

    def run(self):
        self.login_window.show()
        sys.exit(self.app.exec_())

if __name__ == "__main__":
    app = Main()
    app.run()
